幅広くデータを見ていく
* 加速度の波形を並べていみる
* ウィンドウフレームごとの平均，分散の変化

In [3]:
# 表示用
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook

In [4]:
import numpy as np
from numpy import zeros, newaxis
import datetime
import os
import copy
import time

# Parametor about Sensor
SensorNum=1 # Sensor number
#SensorAxis='AccX' # Axis

# Parametor about  " Sliding Window " 
WindowWidth =np.array([128,256,512,1024,2048,4096]) # Window Width
SlidingWidth =WindowWidth/4 #sliding window

# Parametor about Neural Network
AEDimention = 16

# Define Data Name
DATE= str( datetime.date.today() )+'/'
TITLE="Participants01_First_1/"

# Parametor about " Data Path"
DataPath = "/media/takeyama/Transfer/02_ActivityResearch/Paticipants01/01_First/20161018-105301/mem/"

#chopin
#StorePath = "/home/takeyama/Documents/"
#mozart
StorePath ="/media/takeyama/HD-PNFU3/01_ActivityResearchData/"
if not os.path.exists(StorePath+TITLE+'dictionary/'): 
    os.makedirs(StorePath+TITLE+'dictionary/')
DictionaryDataPath=StorePath+TITLE+'dictionary/'

if not os.path.exists(StorePath+TITLE+'window/'): 
    os.makedirs(StorePath+TITLE+'window/')
WindowDataPath=StorePath+TITLE+'window/'

if not os.path.exists(StorePath+TITLE+DATE+'studyOutput/'): 
    os.makedirs(StorePath+TITLE+DATE+'studyOutput/')
StudyOutputPath=StorePath+TITLE+DATE+'studyOutput/'

if not os.path.exists(StorePath+TITLE+DATE+'parametor/'): 
    os.makedirs(StorePath+TITLE+DATE+'parametor/')
ParametorPath=StorePath+TITLE+DATE+'parametor/'

if not os.path.exists(StorePath+TITLE+DATE+'result/'): 
    os.makedirs(StorePath+TITLE+DATE+'result/')
ResultPath=StorePath+TITLE+DATE+'result/'

if not os.path.exists(StorePath+'/'+TITLE+DATE+'graph/'): 
    os.makedirs(StorePath+'/'+TITLE+DATE+'graph/')
GraphPath=StorePath+'/'+TITLE+DATE+'graph/'

センサデータの読み込み

In [5]:
# センサデータのcsvファイル名をリストに集約する関数
def SensorDataFileNameList(path):
    l = os.listdir(DataPath)
    FilePath =[]
    SensorName = []
    
    for n in range(len(l)):
        if (l[n][:4].find('mem-') != -1):
            FilePath.append(l[n])
            SensorName.append(l[n][4:15])
    
    return FilePath,SensorName

CSVを読み込んで，CSVの中身を辞書型で表現する

In [6]:
# FFT library
from scipy import fftpack
from scipy import signal

# Graph Plot
import matplotlib.pyplot as plt
from matplotlib import gridspec

# Generate WindowFrame Function
from mymodule import window

# timer
import time

%matplotlib inline

In [7]:
DictName = os.listdir(DictionaryDataPath)
print DictName

['Chest.npz', 'LeftHand.npz', 'LeftLeg.npz', 'RightHand.npz', 'Rightleg.npz', 'West.npz']


In [8]:
Sensor5 = np.load(DictionaryDataPath+DictName[0])['data'][()]
Sensor1 = np.load(DictionaryDataPath+DictName[1])['data'][()]
Sensor3 = np.load(DictionaryDataPath+DictName[2])['data'][()]
Sensor2 = np.load(DictionaryDataPath+DictName[3])['data'][()]
Sensor4 = np.load(DictionaryDataPath+DictName[4])['data'][()]
Sensor6 = np.load(DictionaryDataPath+DictName[5])['data'][()]

In [9]:
def GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH):
    """
    data　ウィンドウフレームに変換するデータ
    registName　ウィンドウフレームに登録するデータの名前
    windowWidth　ウィンドウ幅
    slidingWidth　スライド幅
    PATH ウィドウフレームを保存するディレクトリ
    
    
    About Function:
        与えられたdataからウィンドウ幅windowWidth,スライド幅slidingWidthにしたがって
        registNameのウィドウフレームを返す．
        また，与えれたPATH内に同様なパラメータ( WindowWidth, slidingWidth)かつ同様な
        windowNameのものがある場合，そのデータを返す．
        この関数が登録，ウィンドウフレームに変換できるデータは１つとする
    """
    storedName = windowName+'_Win='+str(wWidth).zfill(4)+'_Sld='+str(sWidth).zfill(4)+'.npz'
    l = os.listdir(PATH)
    
    if storedName in l:
        print "this data had finished making"
        return np.load(PATH+storedName)['data'][()]
    
    w=window()
    w.SetData(windowName,dataRaw)    
    wind=w.Compile(wWidth,sWidth)
    windoW=wind.reshape((len(wind),np.prod(wind.shape[1:])))
    
    np.savez(PATH+storedName,data=wind)

    return wind

In [10]:
windowNum = 256
slidingNum = windowNum/32

wind=GetWindowFrame('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)

this data had finished making


In [11]:
wind.shape

(134702, 256)

# Methematical/Statistical Functions

## Mean, Median

In [12]:
def Vector_Mean(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    
    Output = np.array([])
    for i in tqdm_notebook( range( wind.shape[0] ) ):
        Output = np.append(Output, wind[i,:].mean())
    return Output

In [13]:
def Vector_Median(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    
    Output = np.array([])
    for i in tqdm_notebook( range( wind.shape[0] ) ):
        Output = np.append(Output, np.median(wind[i,:].mean()) )
    return Output

In [14]:
Vector_Mean('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)

this data had finished making



array([ 0.99480742,  0.99419883,  0.99307266, ..., -0.22953281,
       -0.22897422, -0.22832773])

In [15]:
Vector_Median('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)

this data had finished making



array([ 0.99480742,  0.99419883,  0.99307266, ..., -0.22953281,
       -0.22897422, -0.22832773])

## Variance, Std Deviation

In [16]:
def Vector_Var(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    
    Output = np.array([])
    for i in tqdm_notebook( range( wind.shape[0] ) ):
        Output = np.append(Output, wind[i,:].var())
    return Output

In [17]:
def Vector_StdDev(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    
    Output = np.array([])
    for i in tqdm_notebook( range( wind.shape[0] ) ):
        Output = np.append(Output, wind[i,:].var()**0.5)
    return Output

In [18]:
Vector_Var('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)

this data had finished making



array([ 0.00019847,  0.00019454,  0.00018795, ...,  0.00286388,
        0.00274794,  0.00263213])

In [19]:
Vector_StdDev('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)

this data had finished making



array([ 0.01408788,  0.01394773,  0.01370958, ...,  0.05351527,
        0.05242077,  0.05130431])

## Min, Max, Range
Range = the difference between maximum and minimum sample values

In [20]:
def Vector_Max(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    
    Output = np.array([])
    for i in tqdm_notebook( range( wind.shape[0] ) ):
        Output = np.append(Output, wind[i,:].max())
    return Output

In [21]:
def Vector_Min(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    
    Output = np.array([])
    for i in tqdm_notebook( range( wind.shape[0] ) ):
        Output = np.append(Output, wind[i,:].min())
    return Output

In [22]:
def Vector_Range(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    
    Output = np.array([])
    for i in tqdm_notebook( range( wind.shape[0] ) ):
        Output = np.append(Output, wind[i,:].max()-wind[i,:].min())
    return Output

In [23]:
Vector_Max('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)

this data had finished making



array([ 1.023 ,  1.023 ,  1.023 , ..., -0.0819, -0.0819, -0.0819])

In [24]:
Vector_Min('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)

this data had finished making



array([ 0.9673,  0.9673,  0.9673, ..., -0.3585, -0.3358, -0.3358])

In [25]:
Vector_Range('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)

this data had finished making



array([ 0.0557,  0.0557,  0.0557, ...,  0.2766,  0.2539,  0.2539])

## RMS

In [26]:
wind[0,:].size

256

In [27]:
wind[0,:]**2

array([ 0.99281296,  1.004004  ,  0.99760144,  1.01425041,  1.02758769,
        1.03002201,  1.02070609,  1.022121  ,  1.03408561,  1.03795344,
        1.02860164,  1.03063104,  1.03306896,  1.02860164,  1.02313225,
        1.01969604,  1.02171664,  1.01767744,  1.00781521,  0.99221521,
        0.99760144,  0.99281296,  0.99022401,  0.98346889,  0.98307225,
        0.98069409,  0.96471684,  0.962361  ,  0.95570176,  0.96510976,
        0.95570176,  0.95277121,  0.9604    ,  0.94711824,  0.938961  ,
        0.94128804,  0.94653441,  0.95413824,  0.93566929,  0.93799225,
        0.93857344,  0.94128804,  0.94653441,  0.94458961,  0.93760489,
        0.94458961,  0.94711824,  0.94848121,  0.95609284,  0.95804944,
        0.96510976,  0.95316169,  0.95140516,  0.96707556,  0.97772544,
        0.97042201,  0.97337956,  0.9801    ,  0.97199881,  0.97140736,
        0.9604    ,  0.97101316,  0.97239321,  0.96609241,  0.95902849,
        0.96138025,  0.95374756,  0.95316169,  0.95140516,  0.95

In [28]:
np.sum( wind[0,:]**2/wind[0,:].size )

0.98984027496093763

In [ ]:
def Vector_RMS(windowName, dataRaw, wWidth, sWidth, PATH):
    # get windowFrame
    wind=GetWindowFrame(windowName, dataRaw, wWidth, sWidth, PATH)
    
    Output = np.array([])
    for i in tqdm_notebook( range( wind.shape[0] ) ):
        Output = np.append(Output, (np.sum( wind[i,:]**2)/wind[i,:].size)**0.5 )
    return Output

In [ ]:
Vector_RMS('AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)

## Correlation, Cross-Correlation

In [31]:
def Vector_Corr(windowNameX, dataRawX, windowNameY, dataRawY, wWidth, sWidth, PATH):
    # get windowFrame
    windX=GetWindowFrame(windowNameX, dataRawX, wWidth, sWidth, PATH)
    windY=GetWindowFrame(windowNameY, dataRawY, wWidth, sWidth, PATH)

    Output = np.array([])
    tmpArray = np.array([])
    
    for i in tqdm_notebook( range( windX.shape[0] ) ):
        tmpArray = np.array([])
        for l in tqdm_notebook( range(windY.shape[0]),leave=False):
            tmpArray = np.append(tmpArray, np.corrcoef(windX[i,:],windY[l,:]) ) 
            
        Output = np.append(Output, tmpArray, axis=0)
    return Output  

In [32]:
Vector_Corr('AccX', Sensor1['AccX'],'AccX', Sensor1['AccX'], windowNum, slidingNum, WindowDataPath)

this data had finished making
this data had finished making



KeyboardInterrupt: 

# Signal magnitude area

In [33]:
def Vector_SMA(windowName, dataRawX, dataRawY, dataRawZ, wWidth, sWidth, PATH):
    # get windowFrame
    windX=GetWindowFrame(windowName, dataRawX, wWidth, sWidth, PATH)
    windY=GetWindowFrame(windowName, dataRawY, wWidth, sWidth, PATH)
    windZ=GetWindowFrame(windowName, dataRawZ, wWidth, sWidth, PATH)
    
    # windowFrame roop
    for i in tqdm_notebook( range( windX.shape[0] ) ):
        SqrMag = windX[i,:]**2+windY[i,:]**2+windZ[i,:]**2
        Mag = np.sum( SqrMag**0.5 )

        Output = np.append(Output, Mag)
    return Output  

In [34]:
Vector_Corr('Sensor1', Sensor1['AccX'], Sensor1['AccY'], Sensor1['AccZ'],\
            windowNum, slidingNum, WindowDataPath)

  1%|          | 880/134701 [00:00<00:15, 8789.55it/s]

Sensor1 is registed now
Build Complete
Source shape is(1077870,)
window frames num =134702
offset=6


100%|██████████| 134701/134701 [1:30:10<00:00, 24.90it/s]


TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('S32') dtype('S32') dtype('S32')